Antes de mais nada os notebooks aqui mostrado tiveram como base/foram retirados dos seguintes repositórios: 
 > https://github.com/fchollet/deep-learning-with-python-notebooks 
 
 
 > https://github.com/cdfmlr/Deep-Learning-with-Python-Notebooks
 
 Sugiro fortemente que consultem os códigos originais e em caso de dúvida podem me contatar para conversarmos. 

# Aprendizado profundo com Python

## 8.1 Geração de texto com LSTM

> Use LSTM para gerar texto

Alguém já disse: “gerar dados sequenciais é o mais próximo que os computadores chegam de sonhar.” É muito atraente permitir que os computadores gerem sequências. Tomaremos a geração de texto como exemplo para discutir como usar redes neurais recorrentes para gerar dados de sequência. Essa tecnologia também pode ser usada para geração de música, síntese de voz, geração de diálogo de chatbot e até mesmo escrita de roteiro de filme.

Na verdade, o algoritmo LSTM com o qual estamos familiarizados foi usado para gerar texto caractere por caractere quando foi desenvolvido pela primeira vez.

### Sequência de geração de dados

O método geral de geração de sequências com aprendizado profundo é treinar uma rede (geralmente RNN ou CNN), inserir o Token anterior e prever o próximo Token na sequência.

A terminologia é um pouco: dado o Token anterior, a rede que pode modelar a probabilidade do próximo Token é chamada de "modelo de linguagem". O modelo de linguagem pode capturar a estrutura estatística da linguagem - "espaço latente". Treine um modelo de linguagem, insira a string de texto inicial (chamada de "dados de condicionamento") e faça uma amostra do modelo de linguagem para gerar um novo token, adicione o novo token aos dados condicionais, insira-o novamente e repita o processo Você pode gerar sequências de qualquer comprimento.

Vamos começar com um exemplo simples: use uma camada LSTM, insira uma string de N caracteres do corpus do texto e treine o modelo para gerar o N + 1 º caractere. A saída do modelo é fazer softmax e obter a distribuição de probabilidade do próximo caractere em todos os caracteres possíveis. Este modelo é chamado de "modelo de linguagem neural de nível de caractere".

### Estratégia de amostragem

Ao usar um modelo de linguagem neural em nível de caractere para gerar texto, a questão mais importante é como selecionar o próximo caractere. Aqui estão alguns métodos comuns:

Amostragem -Greedy: sempre selecione o próximo personagem com a maior probabilidade. É provável que esse método obtenha strings repetitivas e previsíveis, e o significado pode não ser consistente. (Associação de método de entrada)
- Amostragem aleatória pura: desenhe o próximo personagem de uma distribuição de probabilidade uniforme, onde cada personagem tem a mesma probabilidade. Essa aleatoriedade é muito alta e quase nenhum conteúdo interessante é gerado. (É uma combinação de caracteres de saída aleatórios)
-Amostragem estocástica: De acordo com os resultados do modelo de linguagem, se a probabilidade do próximo caractere ser e é 0,3, então você tem 30% de probabilidade de escolhê-lo. Existe um pouco de aleatoriedade, o que torna o conteúdo gerado mais ~~ aleatório ~~ rico em variedade, mas não completamente aleatório, a saída pode ser mais interessante.

A amostragem aleatória parece boa e criativa, mas há um problema de que é impossível controlar a magnitude da aleatoriedade: quanto maior a aleatoriedade, mais criativa, mas a saída pode ser aleatória; quanto menor a aleatoriedade, mais próximo das palavras e frases reais, mas também Rígido e previsível.

Para controlar a aleatoriedade durante o processo de amostragem, é introduzido um parâmetro: "softmax temperature" (softmax temperature), usado para representar a entropia da distribuição de probabilidade de amostragem, ou seja, quão inesperado ou previsível será o próximo caractere selecionado:

- Temperatura mais alta: uma distribuição de amostragem com maior entropia gerará dados mais inesperados e não estruturados;
- Menor temperatura: Correspondendo a menos aleatoriedade, dados mais previsíveis serão gerados.

A implementação específica é ponderar novamente a saída softmax do modelo dado o valor da temperatura para obter uma nova distribuição de probabilidade:

In [1]:
import numpy as np

def rewight_distribution(original_distributon, temperature=0.5):
    '''
   Para diferentes temperaturas de softmax, pondere novamente a distribuição de probabilidade
    '''
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

### Implementação de geração de texto LSTM em nível de caractere

A teoria é a acima, agora, precisamos usar Keras para obter a geração de texto LSTM em nível de caractere.

#### preparação de dados

Primeiro, precisamos de uma grande quantidade de dados de texto (corpus) para treinar o modelo de linguagem. Você pode encontrar um ou mais arquivos de texto grandes o suficiente: Wikipedia, vários livros, etc. estão disponíveis. Aqui optamos por utilizar algumas obras de Nietzsche (tradução para o inglês), de modo que o modelo de linguagem que aprendemos tenha o estilo e o tema da escrita de Nietzsche. 

In [2]:
# Baixe o corpus e converta-o para todas as letras minúsculas

from tensorflow import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt', 
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


606208/600901 [==============================] - 430s 709us/step
Corpus length: 600893


Em seguida, temos que transformar o texto em dados (vetorização): extrair a sequência de comprimento `maxlen` do texto (há sobreposição parcial entre as sequências), realizar a codificação one-hot e, em seguida, compactá-la em` (sequências, maxlen, caracteres_únicos) em forma. Ao mesmo tempo, você também precisa preparar uma matriz `y`, que contém o destino correspondente, ou seja, o caractere que aparece após cada sequência extraída (também codificado em um-hot):

In [6]:
# Vetorizar sequências de caracteres

maxlen = 60     # Comprimento de cada sequência
step = 3        # Experimente uma nova sequência a cada 3 caracteres
sentences = []  # Salve a sequência extraída
next_chars = [] # o próximo personagem de frases

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i+maxlen])
    next_chars.append(text[i+maxlen])
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
char_indices = dict((char, chars.index(char)) for char in chars)
# Insira: as duas linhas de código 6 acima
print('Unique characters:', len(chars))

print('Vectorization...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


#### Construa a rede

A rede que vamos usar é muito simples, apenas uma camada LSTM + uma camada Densa ativada por softmax. Na verdade, não é necessário usar LSTM, mas também é possível gerar sequências com camadas convolucionais unidimensionais.

In [7]:
# Modelo LSTM de camada única para prever o próximo personagem

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

In [8]:
# Configuração de compilação de modelo

from tensorflow.keras import optimizers

optimizer = optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)

#### Treine e experimente modelos de linguagem

Dado um modelo de linguagem e um fragmento de texto inicial, o novo texto pode ser gerado repetindo as seguintes operações:

1. Dado o texto existente, obtenha a distribuição de probabilidade do próximo caractere do modelo;
2. Pesar novamente a distribuição de acordo com uma determinada temperatura;
3. Faça uma amostra aleatória do próximo caractere de acordo com a distribuição reponderada;
4. Adicione novos caracteres ao final do texto.

Antes de treinar o modelo, primeiro escrevemos a "função de amostragem":

In [9]:
def sample(preds, temperature=1.0):
    '''
Pesar novamente a distribuição de probabilidade original obtida pelo modelo e extrair um índice de caracteres a partir dela
    '''
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Finalmente, vamos treinar e gerar texto. Usamos uma série de valores de temperatura diferentes para gerar o texto após a conclusão de cada rodada, para que possamos ver como o texto gerado muda conforme o modelo converge e o efeito da temperatura na estratégia de amostragem:

In [17]:
# Loop de geração de texto 
import random

for epoch in range(1, 60):    # Treinamento por 60 rodadas
    print(f'👉\033[1;35m epoch {epoch} \033[0m')    # print('epoch', epoch)
    
    model.fit(x, y,
              batch_size=128,
              epochs=1)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print(f'  📖 Generating with seed: "\033[1;32;43m{generated_text}\033[0m"')    # print(f' Generating with seed: "{generated_text}"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print(f'\n   \033[1;36m 🌡️ temperature: {temperature}\033[0m')    # print('\n  temperature:', temperature)
        print(generated_text, end='')
        for i in range(400):    # Gere 400 caracteres
            # one-hot Codifique o texto atual
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1
            
            # Prever, provar, gerar o próximo personagem
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            print(next_char, end='')
            
            generated_text = generated_text[1:] + next_char
            
    print('\n' + '-' * 20)

👉 epoch 1 
1565/1565 [==============================] - 170s 108ms/step - loss: 1.4089
  📖 Generating with seed: "ary!--will at least be entitled to demand in return that
psy"

    🌡️ temperature: 0.2
ary!--will at least be entitled to demand in return that
psychological senses of the the most comprehensed that is a sense of a perhaps the experience of the heart the present that the profound that the experience of the exploition and present that is a self and the present that is a more of the senses of a more and the sense of a more art and the the exploition and self-contempt of a perhaps the superiom and perhaps the contempt of the superiom and all th
    🌡️ temperature: 0.5
superiom and perhaps the contempt of the superiom and all the instance and plays place of comprehensed and so in morals and all the auther to present mettoral of the senses and fellines to have the conceive that the
possibility of the expendeness, the hasters as how to really expendened and
all that the forenies;

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


sant, who commanent
as those whom them has 'maves for pressonce of list as truth," they are a matter the name is, "that it is noth
    🌡️ temperature: 1.2
t as truth," they are a matter the name is, "that it is nothing, but," all virtuan as his world suxter? is his feetering refuin finger, that is not: its god, als quieus' has
far as ruse is atmashfof, onedy fedufined? will assumphit, "ye willbum'." like
the truiloso
inlogged,--this haid men (personar " logic will"
opinive conectity. at us a
awmetting somewudes ajot, has
being dit wet
beegous. through which only he want clears its sound stands of wishes.


8
--------------------
👉 epoch 19 
1565/1565 [==============================] - 416s 266ms/step - loss: 1.2955
  📖 Generating with seed: "ditioned
(of the metaphysical world) and the world known to "

    🌡️ temperature: 0.2
ditioned
(of the metaphysical world) and the world known to the sense of the sense of the spirit and spirits of the sense of the sense of the best and soul the p

Usando mais dados para treinar um modelo maior, o tempo de treinamento será maior e as amostras geradas serão mais coerentes e realistas. No entanto, o texto gerado desta forma não tem significado. Tudo o que a máquina faz é amostrar dados de modelos estatísticos, ela não entende a linguagem humana nem sabe o que está dizendo.